In [3]:
from datetime import datetime as dt
import requests
import pandas as pd
import Tools as t
import Constants as c

In [4]:
api_key = 'c06e566dfb583f3a48a89143106b6241'
#sport = 'americanfootball_ncaaf'
sport = 'basketball_ncaab'
region = 'us'
market = 'spreads'

In [5]:
def get_odds(sport, market):

    odds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
        'api_key': c.api_key,
        'sport': sport,
        'region': c.region,
        'mkt': market,
    }).json()
    print(odds_response)
    df = pd.DataFrame.from_dict(odds_response['data'])

    df['date'] = df['commence_time'].apply(lambda x: dt.fromtimestamp(x).replace(hour=0, minute=0, tzinfo=None))
    df['left_home'] = df.apply(lambda row: 1 if row.teams[0] == row.home_team else 0, axis = 1)
    df['away_team'] = df.apply(lambda row: row.teams[1] if row.left_home == 1 else row.teams[0], axis = 1)
    df['game'] = df['date'].astype('str') + ' ' + df['away_team'] + ' ' + df['home_team']
    if market == 'spreads':
        bet = 'spread'
    else:
        bet = 'total'

    df[bet] = df.apply(lambda x:  t.most_frequent([i['odds'][market]['points'][1] for i in x.sites]) if ( x.home_team == 0) else t.most_frequent([i['odds'][market]['points'][0] for i in x.sites]), axis=1)
    df.drop(['left_home'], axis=1, inplace=True)
    return df


In [6]:
df = get_odds(sport='basketball_nba', market='spreads')
df.head(5)

{'success': True, 'data': [{'sport_key': 'basketball_nba', 'sport_nice': 'NBA', 'teams': ['Atlanta Hawks', 'Los Angeles Clippers'], 'commence_time': 1611708000, 'home_team': 'Atlanta Hawks', 'sites': [{'site_key': 'fanduel', 'site_nice': 'FanDuel', 'last_update': 1611698779, 'odds': {'spreads': {'odds': [1.88, 1.94], 'points': ['-5.0', '5.0']}}}, {'site_key': 'betmgm', 'site_nice': 'BetMGM', 'last_update': 1611698801, 'odds': {'spreads': {'odds': [1.95, 1.87], 'points': ['-5.5', '5.5']}}}, {'site_key': 'betrivers', 'site_nice': 'BetRivers', 'last_update': 1611698874, 'odds': {'spreads': {'odds': [1.89, 1.92], 'points': ['-5.0', '5.0']}}}, {'site_key': 'draftkings', 'site_nice': 'DraftKings', 'last_update': 1611698858, 'odds': {'spreads': {'odds': [1.89, 1.92], 'points': ['-5.0', '5.0']}}}, {'site_key': 'unibet', 'site_nice': 'Unibet', 'last_update': 1611698914, 'odds': {'spreads': {'odds': [1.9, 1.92], 'points': ['-5.0', '5.0']}}}, {'site_key': 'sugarhouse', 'site_nice': 'SugarHouse', 

,sport_key,sport_nice,teams,commence_time,home_team,sites,sites_count,date,away_team,game,spread
0,basketball_nba,NBA,"[Atlanta Hawks, Los Angeles Clippers]",1611708000,Atlanta Hawks,"[{'site_key': 'fanduel', 'site_nice': 'FanDuel...",13,2021-01-26,Los Angeles Clippers,2021-01-26 Los Angeles Clippers Atlanta Hawks,-5.0
1,basketball_nba,NBA,"[Houston Rockets, Washington Wizards]",1611709800,Houston Rockets,"[{'site_key': 'fanduel', 'site_nice': 'FanDuel...",13,2021-01-26,Washington Wizards,2021-01-26 Washington Wizards Houston Rockets,-3.5
2,basketball_nba,NBA,"[New York Knicks, Utah Jazz]",1611713400,Utah Jazz,"[{'site_key': 'caesars', 'site_nice': 'Caesars...",13,2021-01-26,New York Knicks,2021-01-26 New York Knicks Utah Jazz,11.5
3,basketball_nba,NBA,"[Cleveland Cavaliers, Detroit Pistons]",1611792600,Cleveland Cavaliers,"[{'site_key': 'betonlineag', 'site_nice': 'Bet...",7,2021-01-27,Detroit Pistons,2021-01-27 Detroit Pistons Cleveland Cavaliers,-2.5
4,basketball_nba,NBA,"[Orlando Magic, Sacramento Kings]",1611792600,Orlando Magic,"[{'site_key': 'betonlineag', 'site_nice': 'Bet...",7,2021-01-27,Sacramento Kings,2021-01-27 Sacramento Kings Orlando Magic,-1.0


In [9]:
df2 = get_odds(sport='basketball_nba', market='totals')
df2.head(5)

{'success': True, 'data': [{'sport_key': 'basketball_nba', 'sport_nice': 'NBA', 'teams': ['Charlotte Hornets', 'Chicago Bulls'], 'commence_time': 1611360600, 'home_team': 'Charlotte Hornets', 'sites': [{'site_key': 'pointsbetus', 'site_nice': 'PointsBet (US)', 'last_update': 1611352467, 'odds': {'totals': {'points': [228.0, 228.0], 'odds': [1.87, 1.95], 'position': ['over', 'under']}}}, {'site_key': 'betmgm', 'site_nice': 'BetMGM', 'last_update': 1611352575, 'odds': {'totals': {'points': ['227.5', '227.5'], 'odds': [1.91, 1.91], 'position': ['over', 'under']}}}, {'site_key': 'unibet', 'site_nice': 'Unibet', 'last_update': 1611352463, 'odds': {'totals': {'points': [228.0, 228.0], 'odds': [1.89, 1.93], 'position': ['over', 'under']}}}, {'site_key': 'betrivers', 'site_nice': 'BetRivers', 'last_update': 1611352573, 'odds': {'totals': {'points': ['228.0', '228.0'], 'odds': [1.88, 1.93], 'position': ['over', 'under']}}}, {'site_key': 'sugarhouse', 'site_nice': 'SugarHouse', 'last_update': 16

,sport_key,sport_nice,teams,commence_time,home_team,sites,sites_count,date,away_team,game,total
0,basketball_nba,NBA,"[Charlotte Hornets, Chicago Bulls]",1611360600,Charlotte Hornets,"[{'site_key': 'pointsbetus', 'site_nice': 'Poi...",13,2021-01-22,Chicago Bulls,2021-01-22 Chicago Bulls Charlotte Hornets,228.0
1,basketball_nba,NBA,"[Indiana Pacers, Orlando Magic]",1611360600,Indiana Pacers,"[{'site_key': 'pointsbetus', 'site_nice': 'Poi...",13,2021-01-22,Orlando Magic,2021-01-22 Orlando Magic Indiana Pacers,217.5
2,basketball_nba,NBA,"[Detroit Pistons, Houston Rockets]",1611360600,Detroit Pistons,"[{'site_key': 'pointsbetus', 'site_nice': 'Poi...",13,2021-01-22,Houston Rockets,2021-01-22 Houston Rockets Detroit Pistons,215.0
3,basketball_nba,NBA,"[Brooklyn Nets, Cleveland Cavaliers]",1611362400,Cleveland Cavaliers,"[{'site_key': 'williamhill_us', 'site_nice': '...",12,2021-01-22,Brooklyn Nets,2021-01-22 Brooklyn Nets Cleveland Cavaliers,223.0
4,basketball_nba,NBA,"[Boston Celtics, Philadelphia 76ers]",1611362400,Philadelphia 76ers,"[{'site_key': 'pointsbetus', 'site_nice': 'Poi...",12,2021-01-22,Boston Celtics,2021-01-22 Boston Celtics Philadelphia 76ers,221.5


In [ ]:
live_game = df[(df['away_team'].str.lower() == 'Buffalo Bulls'.lower()) & (df['home_team'].str.lower() == 'Miami (OH) Redhawks'.lower())]
live_game['total'].values[0]

In [ ]:
df['home_team'].head()

In [1]:
odds_response = requests.get('https://api.the-odds-api.com/v3/sports', params={
    'all':True,
    'api_key': api_key,
}).json()
df = pd.DataFrame.from_dict(odds_response['data'])
df

NameError: name 'requests' is not defined

In [7]:
df['teams'].iloc[0]


['Atlanta Hawks', 'Los Angeles Clippers']

In [11]:
pd.notnull(df['teams'].iloc[0])

array([ True,  True])

In [5]:
df.iloc[0].to_list()

['1/1/1900 0:00', 1, '0:20:00', 37, 29, 66]

In [9]:
df.first_valid_index()

'1 0:01'

In [6]:
'FinalOT'.startswith('Final')

True

In [29]:
for i, col in enumerate(['a', 'b', 'c'][1:], start=1):
    print(i, col)

1 b
2 c


In [5]:
import pandas as pd
df2 = pd.read_csv('player_pbp_test.csv')
df2 = df2[df2['player'].notnull()]

In [6]:
df2.tail()

,time_stamp,time,period,adj_time,away,home,total,play,player,points,team
174,1 10:32 6 3,1900-01-01 00:10:32,1,0:01:28,6,3,9,Andre Drummond makes free throw 1 of 2,A Drummond,1.0,Cleveland Cavaliers
177,1 10:49 6 2,1900-01-01 00:10:49,1,0:01:11,6,2,8,Mason Plumlee makes 2-foot dunk (Delon Wright ...,M Plumlee,2.0,Detroit Pistons
179,1 11:11 4 2,1900-01-01 00:11:11,1,0:00:49,4,2,6,Wayne Ellington makes 25-foot three point shot...,W Ellington,3.0,Detroit Pistons
180,1 11:24 1 2,1900-01-01 00:11:24,1,0:00:36,1,2,3,Wayne Ellington makes technical free throw,W Ellington,1.0,Detroit Pistons
182,1 11:44 0 2,1900-01-01 00:11:44,1,0:00:16,0,2,2,Andre Drummond makes driving layup,A Drummond,2.0,Cleveland Cavaliers


In [7]:
one_hot = pd.get_dummies(df2['period'])
one_hot = one_hot.T.reindex([1, 2, 3, 4]).T.fillna(0)

In [8]:
one_hot.tail()

,1,2,3,4
174,1.0,0.0,0.0,0.0
177,1.0,0.0,0.0,0.0
179,1.0,0.0,0.0,0.0
180,1.0,0.0,0.0,0.0
182,1.0,0.0,0.0,0.0


In [9]:
quarters = list(range(1, 4))
quarters

[1, 2, 3]

In [10]:
one_hot= one_hot[[1, 2, 3, 4]].multiply(df2['points'], axis='index')

In [11]:
one_hot.tail()


,1,2,3,4
174,1.0,0.0,0.0,0.0
177,2.0,0.0,0.0,0.0
179,3.0,0.0,0.0,0.0
180,1.0,0.0,0.0,0.0
182,2.0,0.0,0.0,0.0


In [12]:
player_df = pd.concat([df2, one_hot], axis=1)

In [13]:
player_df = player_df[['player', 'points', 'team', 1, 2, 3, 4]]

In [14]:
player_df.head()

,player,points,team,1,2,3,4
1,A Drummond,1.0,Cleveland Cavaliers,0.0,1.0,0.0,0.0
2,A Drummond,2.0,Cleveland Cavaliers,0.0,2.0,0.0,0.0
11,D Windler,2.0,Cleveland Cavaliers,0.0,2.0,0.0,0.0
12,M Plumlee,1.0,Detroit Pistons,0.0,1.0,0.0,0.0
14,A Drummond,2.0,Cleveland Cavaliers,0.0,2.0,0.0,0.0


In [15]:
grouped = player_df.groupby(['player','team'], as_index=False).sum()
grouped

,player,team,points,1,2,3,4
0,A Drummond,Cleveland Cavaliers,12.0,7.0,5.0,0.0,0.0
1,C Osman,Cleveland Cavaliers,3.0,0.0,3.0,0.0,0.0
2,C Sexton,Cleveland Cavaliers,16.0,8.0,8.0,0.0,0.0
3,D Dotson,Cleveland Cavaliers,2.0,0.0,2.0,0.0,0.0
4,D Garland,Cleveland Cavaliers,9.0,9.0,0.0,0.0,0.0
5,D Rose,Detroit Pistons,4.0,4.0,0.0,0.0,0.0
6,D Windler,Cleveland Cavaliers,2.0,0.0,2.0,0.0,0.0
7,D Wright,Detroit Pistons,7.0,2.0,5.0,0.0,0.0
8,I Okoro,Cleveland Cavaliers,2.0,2.0,0.0,0.0,0.0
9,J Allen,Cleveland Cavaliers,4.0,2.0,2.0,0.0,0.0


In [165]:
away = 'Cleveland Cavaliers'
home = 'Detroit Pistons'
order = pd.CategoricalDtype([away, home], ordered=True)

In [166]:
grouped['team'] = grouped['team'].astype(order)
grouped.sort_values('team', inplace=True)

,player,team,points,1,2,3,4
0,A Drummond,Cleveland Cavaliers,12.0,7.0,5.0,0.0,0.0
9,J Allen,Cleveland Cavaliers,4.0,2.0,2.0,0.0,0.0
15,T Prince,Cleveland Cavaliers,4.0,4.0,0.0,0.0,0.0
6,D Windler,Cleveland Cavaliers,2.0,0.0,2.0,0.0,0.0
8,I Okoro,Cleveland Cavaliers,2.0,2.0,0.0,0.0,0.0
3,D Dotson,Cleveland Cavaliers,2.0,0.0,2.0,0.0,0.0
2,C Sexton,Cleveland Cavaliers,16.0,8.0,8.0,0.0,0.0
1,C Osman,Cleveland Cavaliers,3.0,0.0,3.0,0.0,0.0
4,D Garland,Cleveland Cavaliers,9.0,9.0,0.0,0.0,0.0
5,D Rose,Detroit Pistons,4.0,4.0,0.0,0.0,0.0


In [167]:
grouped['site'] = grouped['team'].apply(lambda x: 1 if x == home else ( 0 if x == away else x))


In [168]:
grouped

,player,team,points,1,2,3,4,site
0,A Drummond,Cleveland Cavaliers,12.0,7.0,5.0,0.0,0.0,0
1,C Osman,Cleveland Cavaliers,3.0,0.0,3.0,0.0,0.0,0
2,C Sexton,Cleveland Cavaliers,16.0,8.0,8.0,0.0,0.0,0
3,D Dotson,Cleveland Cavaliers,2.0,0.0,2.0,0.0,0.0,0
4,D Garland,Cleveland Cavaliers,9.0,9.0,0.0,0.0,0.0,0
5,D Rose,Detroit Pistons,4.0,4.0,0.0,0.0,0.0,1
6,D Windler,Cleveland Cavaliers,2.0,0.0,2.0,0.0,0.0,0
7,D Wright,Detroit Pistons,7.0,2.0,5.0,0.0,0.0,1
8,I Okoro,Cleveland Cavaliers,2.0,2.0,0.0,0.0,0.0,0
9,J Allen,Cleveland Cavaliers,4.0,2.0,2.0,0.0,0.0,0


In [4]:
import pandas as pd
g2 = pd.read_csv('g2.csv')
g2

,Unnamed: 0,player,team,points,1,2,site
0,8,Hassan Whiteside,Sacramento Kings,12,6,6,0
1,1,Buddy Hield,Sacramento Kings,10,4,6,0
2,3,Cory Joseph,Sacramento Kings,6,3,3,0
3,4,De'Aaron Fox,Sacramento Kings,12,5,7,0
4,13,Richaun Holmes,Sacramento Kings,3,0,3,0
5,7,Harrison Barnes,Sacramento Kings,15,13,2,0
6,15,Tyrese Haliburton,Sacramento Kings,3,0,3,0
7,10,Marvin Bagley III,Sacramento Kings,7,5,2,0
8,14,Terence Davis,Toronto Raptors,10,2,8,1
9,12,Pascal Siakam,Toronto Raptors,17,9,8,1


In [5]:
g2.drop(g2.columns[0], axis=1, inplace=True)
g2.iloc[0].to_list()

['Hassan Whiteside', 'Sacramento Kings', 12, 6, 6, 0]

In [6]:
g2

,player,team,points,1,2,site
0,Hassan Whiteside,Sacramento Kings,12,6,6,0
1,Buddy Hield,Sacramento Kings,10,4,6,0
2,Cory Joseph,Sacramento Kings,6,3,3,0
3,De'Aaron Fox,Sacramento Kings,12,5,7,0
4,Richaun Holmes,Sacramento Kings,3,0,3,0
5,Harrison Barnes,Sacramento Kings,15,13,2,0
6,Tyrese Haliburton,Sacramento Kings,3,0,3,0
7,Marvin Bagley III,Sacramento Kings,7,5,2,0
8,Terence Davis,Toronto Raptors,10,2,8,1
9,Pascal Siakam,Toronto Raptors,17,9,8,1


In [7]:
g2['player'].iloc[0]

'Hassan Whiteside'

In [8]:
g2.reset_index()

,index,player,team,points,1,2,site
0,0,Hassan Whiteside,Sacramento Kings,12,6,6,0
1,1,Buddy Hield,Sacramento Kings,10,4,6,0
2,2,Cory Joseph,Sacramento Kings,6,3,3,0
3,3,De'Aaron Fox,Sacramento Kings,12,5,7,0
4,4,Richaun Holmes,Sacramento Kings,3,0,3,0
5,5,Harrison Barnes,Sacramento Kings,15,13,2,0
6,6,Tyrese Haliburton,Sacramento Kings,3,0,3,0
7,7,Marvin Bagley III,Sacramento Kings,7,5,2,0
8,8,Terence Davis,Toronto Raptors,10,2,8,1
9,9,Pascal Siakam,Toronto Raptors,17,9,8,1


In [1]:
from selenium import webdriver, common
from selenium.webdriver.firefox.options import Options
driver = webdriver.Firefox()
driver.get('https://www.espn.com/mens-college-basketball/playbyplay?gameId=401265083')

In [67]:
elements = driver.find_elements_by_class_name('accordion-header')
for e in elements:
    try:
        period_accordion = e.find_element_by_class_name('collapsed')
        if period_accordion.get_attribute('aria-expanded') == 'false':
            period_accordion.click()
    except common.exceptions.NoSuchElementException:
        pass

In [10]:
g2[g2['player'] == 'Hassan Whiteside']

,player,team,points,1,2,site
0,Hassan Whiteside,Sacramento Kings,12,6,6,0
